In [1]:
import sys

sys.path.append('../')

import os
import shutil

if os.path.exists('./runs/booking'):
    shutil.rmtree('./runs/booking')


os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'

In [2]:
from Model import Model_1 as Model

import torch
import torch.nn as nn
import torch.nn.functional as F

from tqdm import tqdm

from torch.utils.tensorboard import SummaryWriter

from sklearn.model_selection import KFold

writer = SummaryWriter('./runs/booking')

torch.autograd.set_detect_anomaly(True)

2024-02-05 21:16:32.017565: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-05 21:16:32.017618: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-05 21:16:32.018737: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-05 21:16:32.025966: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-05 21:16:32.757210: W tensorflow/compiler/tf2

In [3]:
BATCH_SIZE = 1024
EPOCHS = 10
LEARNING_RATE = 0.001
FOLDS = 10
K_ACCURACY = 4
LRS_GAMMA = 0.1
USE_LRS = False

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.backends.cudnn.enabled = True

In [4]:
import random
import numpy as np

def seed_torch(seed_value):
    random.seed(seed_value) # Python
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
    if torch.backends.cudnn.is_available:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

seed_torch(0)

In [5]:
from torch.utils.data import SubsetRandomSampler, DataLoader
from Dataset import PartDataset

X = torch.load('X.pt')
print(X.shape)
y_city = torch.load('y_city.pt')

dataset = PartDataset(X, y_city)

torch.Size([2501964, 20])


In [6]:
def reset_weights(m):
    for layer in m.children():
        if hasattr(layer, 'reset_parameters'):
            layer.reset_parameters()

In [7]:
def accuracy_at_k(outputs, labels, k = 1):
    batch_size = labels.size(0)

    _, pred_indices = outputs.topk(k, 1, True, True)
    correct = torch.sum(torch.argmax(
        labels, dim=1).view(-1, 1) == pred_indices)

    accuracy = correct.item() / batch_size
    return accuracy * 100, correct.item()

In [8]:
def test(test_loader: torch.utils.data.DataLoader, model: nn.Module, criterion: nn.Module):
    loss = 0
    total_correct = 0
    total_samples = 0

    test_loader = tqdm(test_loader, desc='Testing')
    
    with torch.no_grad():
        model.eval()
        for inputs, labels in test_loader:
            inputs = torch.unsqueeze(inputs, 2)

            inputs = inputs.to(torch.float32).to(device)
            labels = labels.to(torch.int64)

            labels = nn.functional.one_hot(labels, num_classes=11989).to(torch.float32).to(device)

            outputs = model(inputs)

            loss += criterion(outputs, labels).item()

            _, correct = accuracy_at_k(outputs, labels, K_ACCURACY)
            total_correct += correct
            total_samples += labels.size(0)

    loss = loss / len(test_loader)
    accuracy = total_correct / total_samples * 100

    return loss, accuracy

In [9]:
def train(epochs: int, train_loader: torch.utils.data.DataLoader, model: nn.Module, optimizer: torch.optim.Optimizer, criterion: nn.Module, save_model: bool = False, lrs: torch.optim.lr_scheduler = None) -> None:
    for epoch in range(epochs):
        model.train()

        train_loader = tqdm(train_loader, desc='Training')

        running_loss = 0.0
        total_correct = 0
        batch = 0
        total_samples = 0

        for i, (inputs, labels) in enumerate(train_loader, 1):
            inputs = torch.unsqueeze(inputs, 2)

            inputs = inputs.to(torch.float32).to(device)
            labels = labels.to(torch.int64)

            labels = nn.functional.one_hot(labels, num_classes=11989).to(torch.float32).to(device)

            outputs = model(inputs)

            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            _, correct = accuracy_at_k(outputs, labels, K_ACCURACY)
            total_correct += correct
            total_samples += labels.size(0)
            running_loss += loss.item()

            if i % 100 == 0:
                epoch_loss = running_loss / i
                batch_accuracy = total_correct / total_samples * 100

                writer.add_scalar(f'Loss/train/batch/{epoch}', epoch_loss, batch)
                writer.add_scalar(f'Accuracy/train/batch/{epoch}', batch_accuracy, batch)

                batch += 1

                writer.flush()

        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = total_correct / total_samples * 100

        print(f"Epoch {epoch} loss: {epoch_loss:.4f}, accuracy: {epoch_accuracy:.2f}%")

        writer.add_scalar('Loss/train/epoch', epoch_loss, epoch)
        writer.add_scalar('Accuracy/train/epoch', epoch_accuracy, epoch)

        writer.flush()

        if save_model:
            torch.save(model.state_dict(), f'./model_{epoch}.pt')

        if lrs is not None:
            lrs.step()

In [10]:
def k_fold_cv(k: int, dataset: torch.utils.data.Dataset, model: nn.Module, optimizer: torch.optim.Optimizer, criterion: nn.Module, lrs: torch.optim.lr_scheduler = None):
    folds = KFold(n_splits=k, shuffle=True)

    for fold, (train_ids, test_ids) in enumerate(folds.split(dataset), 1):
        print(f"Fold {fold}")
        
        model.apply(reset_weights)

        train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, sampler=SubsetRandomSampler(train_ids))
        test_loader = DataLoader(dataset, batch_size=BATCH_SIZE, sampler=SubsetRandomSampler(test_ids))

        for epoch in range(EPOCHS):
            train(1, train_loader, model, optimizer, criterion, False, lrs)

            loss, accuracy = test(test_loader, model, criterion)

            print(F"Epoch {epoch} loss: {loss:.4f}, accuracy: {accuracy:.2f}% in test set")

        writer.add_scalar('Loss/test/fold', loss, fold)
        writer.add_scalar('Accuracy/test/fold', accuracy, fold)

        writer.flush()

        torch.save(model.state_dict(), f'./model_fold_{fold}.pt')

In [11]:
model = Model().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()
lrs = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=LRS_GAMMA) if USE_LRS else None

In [12]:
k_fold_cv(FOLDS, dataset, model, optimizer, criterion, lrs)
# train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)
# train(EPOCHS, train_loader, model, optimizer, criterion, True, lrs)

Fold 1


Training: 100%|██████████| 2199/2199 [12:54<00:00,  2.84it/s]


Epoch 0 loss: 5.5981, accuracy: 24.12%


Testing: 100%|██████████| 245/245 [00:29<00:00,  8.29it/s]


Epoch 0 loss: 4.3743, accuracy: 36.93% in test set


...

Epoch 0 loss: 5.4965, accuracy: 24.30%


Testing: 100%|██████████| 245/245 [00:30<00:00,  8.16it/s]


Epoch 1 loss: 4.5880, accuracy: 33.52% in test set


Training:  20%|█▉        | 429/2199 [02:36<10:47,  2.73it/s]


KeyboardInterrupt: 